### 这个 notebook 用来读取文件，并测试一些方法

In [1]:
import os
import numpy as np

def get_all_py_files(root_dir: str) -> list:
    """
    获取指定目录及其子目录下的所有 .py 文件路径。

    Args:
        root_dir (str): 起始目录路径。

    Returns:
        list: 包含所有 .py 文件完整路径的列表。
    """
    py_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if file.endswith(".py"):
                py_files.append(os.path.join(dirpath, file))
    return py_files

def write_all_py_contents_to_output(py_files: list, output_path: str = "src.txt") -> None:
    """
    将所有 .py 文件的内容写入一个文本文件中，并打印文件名作为分隔。

    Args:
        py_files (list): .py 文件路径列表。
        output_path (str): 输出文件路径。
    """
    with open(output_path, "w", encoding="utf-8") as out_file:
        for path in py_files:
            out_file.write(f"{'=' * 80}\n")
            out_file.write(f"File: {path}\n")
            out_file.write(f"{'-' * 80}\n")
            try:
                with open(path, "r", encoding="utf-8") as f:
                    out_file.write(f.read())
            except Exception as e:
                out_file.write(f"⚠️ Error reading {path}: {e}\n")
            out_file.write("\n\n")

if True:
    root_directory = "."  # 当前目录
    all_py_files = get_all_py_files(root_directory)
    write_all_py_contents_to_output(all_py_files)
    print(f"📄 所有 Python 文件内容已写入 src.txt（共 {len(all_py_files)} 个文件）")



📄 所有 Python 文件内容已写入 src.txt（共 10 个文件）


### 检查 tokenizer

In [2]:
from tokenizer import AlphaTokenizer

tok = AlphaTokenizer()

expr = "open 5 ts_std"          # 相当于 ts_mean(close, 5)

ids = tok.encode(expr)
print(ids)        # ➜ [BOS]  对应 id,  ... , [SEP]

print(tok.decode(ids))


[1, 3, 10, 28, 2]
open 5 ts_std


### 1.测试 AlphaCombinationModel._compute_alpha_from_expr

In [3]:
from combination import AlphaCombinationModel
from data import load_market_data

df = load_market_data()
model = AlphaCombinationModel()
model.inject_data(df, target_col='target')

# 表达式：close 的 5 日均值
expr = "close 5 ts_mean"
alpha = model._compute_alpha_from_expr(expr)

print("alpha shape:", alpha.shape)
print("alpha sample:", alpha[~np.isnan(alpha)][:5])


KeyError: 'close'

###  2. 测试 AlphaCombinationModel.add_alpha_expr

In [ ]:
ic = model.add_alpha_expr("high low - 10 ts_max")
print("该因子的 IC 为：", ic)
print("当前池中因子数：", len(model.alphas))


### 3. 测试 AlphaTokenizer.encode / decode

### 4. 测试 AlphaGenerationEnv.reset / step

### 5. 测试 PolicyNetwork / ValueNetwork 输出维度

### 6. 测试 RLAlphaGenerator._collect_trajectories